In [12]:
from UserItemData import UserItemData
from MovieData import MovieData
from RandomPredictor import RandomPredictor
from Recommender import Recommender
from AveragePredictor import AveragePredictor
from ViewsPredictor import ViewsPredictor
from STDPredictor import STDPredictor
from ItemBasedPredictor import ItemBasedPredictor
from ItemBasedPredictor1 import ItemBasedPredictor1

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 1 Branje ocen
""" uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())
uim = UserItemData('data/user_ratedmovies.dat',  '12.1.2007', '16.2.2008', 100)
print(uim.nratings()) """
#855598
#32697

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 2 Branje filmov
""" md = MovieData('data/movies.dat')
print(md.get_title(1)) """
#Toy story

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 3 Naključni prediktor
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item])) """
#<class 'dict'>
#Film: Toy story, ocena: 3
#Film: Grumpy Old Men, ocena: 5
#Film: Money Train, ocena: 2
#Film: The Usual Suspects, ocena: 1
#Film: City Hall, ocena: 1

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 4 Priporočanje
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """
#Film: Desperado, ocena: 5
#Film: Die Hard: With a Vengeance, ocena: 5
#Film: The Crow, ocena: 5
#Film: Beverly Hills Cop III, ocena: 5
#Film: South Park: Bigger Longer & Uncut, ocena: 5

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 5 Napovedovanje s povprečjem
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
avg_predictor = AveragePredictor(b=100)
avg_predictor.fit(uim)
predictions = avg_predictor.predict(78)
for movie_id, predicted_rating in predictions:
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), predicted_rating)) """
#AveragePredictor(b=0)
#Film: Judge Dredd, ocena: 5
#Film: DragonHeart, ocena: 5
#Film: Last Man Standing, ocena: 5

#AveragePredictor(b=100)
#Film: The Usual Suspects, ocena: 4.225944245560473
#Film: The Godfather: Part II, ocena: 4.146907937910189
#Film: Cidade de Deus, ocena: 4.116538340205236

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-. 

# 6 Priporočanje najbolj gledanih filmov
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
views_predictor = ViewsPredictor()
views_predictor.fit(uim)
predictions = views_predictor.predict()
for movie_id, view_count in predictions:
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), view_count)) """
#Film: The Matrix, ocena: 1670
#Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
#Film: Forrest Gump, ocena: 1568

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 7 Priporočanje kontroverznih filmov
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = STDPredictor(100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """
#Film: Plan 9 from Outer Space, ocena: 1.3386525190884058
#Film: The Passion of the Christ, ocena: 1.2790640858537328
#Film: The Texas Chainsaw Massacre, ocena: 1.2306963889088796
#Film: Jackass Number Two, ocena: 1.2134235427973514
#Film: White Chicks, ocena: 1.184167557494643

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

""" import csv
from collections import defaultdict

def count_movies_rated_by_users(file_path):
    # Dictionary to store user ID and their corresponding count of rated movies
    user_movie_counts = defaultdict(int)
    
    # Open and read the file
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')  # Adjust delimiter if needed
        
        # Skip the header
        next(reader, None)
        
        # Process each row in the file
        for row in reader:
            user_id = int(row[0])  # Assuming first column is user ID
            user_movie_counts[user_id] += 1
    
    # Print the results
    for user_id, movie_count in user_movie_counts.items():
        if user_id==78:
            print(f"User {user_id} rated {movie_count} movies.")

#count_movies_rated_by_users('data/user_ratedmovies.dat') """

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 1
#   42059      42104      
#user_item_matrix = rec.predictor.user_item_matrix
#print(user_item_matrix)
#2571 356
#Movie 2571 has similarities with: [356]
#Movie 356 has similarities with: [2571]
""" md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))
print("Predictions for 42059: ") #42059      42104   
rec_items = rec.recommend(42059  , n=15, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) """
#Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716):  0.3526686233651821
#Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527):  0
#Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780):  0.5112426907698497
#Predictions for 42059: 
#Film: Twelve Monkeys, ocena: 4.5
#Film: Shichinin no samurai, ocena: 4.5
# .....

# -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.

# 2
# Inicializiramo potrebne objekte
""" md = MovieData('data/movies.dat')  # MovieData za naslove filmov
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)  # UserItemData za uporabniške ocene
rp = ItemBasedPredictor1(min_values=1000)  # ItemBasedPredictor za izračun podobnosti
rp.fit(uim)
rp.print_top_20_similar_movies(md) """
#Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.872658016361454
#Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.872658016361454
#Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.8431367104516457
#Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.8431367104516457
#Film1: Austin Powers: The Spy Who Shagged Me, Film2: Catch Me If You Can, podobnost: 0.8398089920602765
#Film1: Catch Me If You Can, Film2: Austin Powers: The Spy Who Shagged Me, podobnost: 0.8398089920602765
# ....




Film1: Pretty Woman, Film2: Catch Me If You Can, podobnost: 0.9504147565678164
Film1: Catch Me If You Can, Film2: Pretty Woman, podobnost: 0.9504147565678164
Film1: Speed, Film2: Pretty Woman, podobnost: 0.9431553466375703
Film1: Pretty Woman, Film2: Speed, podobnost: 0.9431553466375703
Film1: The Matrix Reloaded, Film2: Pretty Woman, podobnost: 0.9247579414374417
Film1: Pretty Woman, Film2: The Matrix Reloaded, podobnost: 0.9247579414374417
Film1: Titanic, Film2: Pretty Woman, podobnost: 0.912961989162464
Film1: Pretty Woman, Film2: Titanic, podobnost: 0.912961989162464
Film1: Terminator Salvation, Film2: Rain Man, podobnost: 0.8788571179625343
Film1: Rain Man, Film2: Terminator Salvation, podobnost: 0.8788571179625343
Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.872658016361454
Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.872658016361454
Film1: The L